# Optimization in each category of dct amplitudes

In [1]:
import os
import cv2
import random
import numpy as np
from mip import Model, xsum, maximize, BINARY

Using Python-MIP package version 1.7.3


In [2]:
def mip_entropy_constrained(Lambda):
    Omega = 1
    p = Omega * all_dijs + Lambda * all_prb0s
    w = A_eq
    c = 1
    I = range(np.shape(w)[1])

    m = Model('knapsack')

    x = [m.add_var(var_type=BINARY) for i in I]

    m.objective = maximize(xsum(p[i] * x[i] for i in I))

    for j in range (0,cells_num):
        m += xsum(w[j][i] * x[i] for i in I) == c

    opt_value = m.optimize()

    # print('\n', mip_pairs_show)
    selected = [i for i in I if x[i].x >= 0.99]
    # print('selected items: {}'.format(selected))

    mip_pairs = []
    for i in I:
        mip_pair = x[i].x
        mip_pairs = np.append(mip_pairs, mip_pair)

    k = 0
    mip_xx = np.zeros([cells_num, cells_num])
    for i in range(0,cells_num):
        for j in range(i+1,cells_num):
            mip_xx[i,j] = mip_pairs[k]
            k +=1

    mip_pairs_show = np.array(np.where(mip_xx == 1))

    obj_value_db = 10*np.log10(m.objective_value)
    
    mip_dijs = mip_pairs * all_dijs
    mip_distortion = sum(mip_dijs)
    mip_db = 10 * np.log10(mip_distortion)

    mip_prb0s = mip_pairs * all_prb0s
    mip_prb0 = sum(mip_prb0s)
    if mip_prb0 == 0 or mip_prb0 == 1:
        mip_entropy = 0
    else:
        mip_entropy = -mip_prb0 * np.log2(mip_prb0)-(1-mip_prb0)*np.log2(1-mip_prb0)

    # print(opt_distortion)
    print('\n1_DE(dB)_lam'+str(Lambda),mip_db)
    print('1_H(S0)_lam'+str(Lambda), mip_entropy)
     
    dual_mip_xx = mip_xx + np.transpose(mip_xx)
       
       
    # At Eve ######### ######### ######### ######### ######### ######### ######### #########
#     # 1 MSB Encryption
#     msb_pair1 = mip_pairs_show[0]
#     msb_pair2 = mip_pairs_show[1]

#     eve_msb0 = intensities.copy()
#     for i in range(0,height):
#         for j in range(0,width):
#             paired_cell = np.where(dual_mip_xx[intensities[i][j]] == 1)
#             if prbs[intensities[i][j]] < prbs[paired_cell]:
#                 eve_msb0[i][j] = paired_cell[0]
#     cv2.imshow('1_msb0_lam'+str(Lambda), eve_msb0)
#     filename = '1_msb0_lam'+str(Lambda)+'.webp'
#     cv2.imwrite(filename, eve_msb0)
#     cv2.waitKey(1000)
        
#     eve_msb1 = intensities.copy()
#     for i in range(0,height):
#         for j in range(0,width):
#             paired_cell = np.where(dual_mip_xx[intensities[i][j]] == 1)
#             if prbs[intensities[i][j]] > prbs[paired_cell]:
#                 eve_msb1[i][j] = paired_cell[0]

#     cv2.imshow('1_msb1_lam'+str(Lambda), eve_msb1)
#     filename = '1_msb1_lam'+str(Lambda)+'.webp'
#     cv2.imwrite(filename, eve_msb1)
#     cv2.waitKey(1000)
        
    return mip_pairs_show, mip_db, mip_entropy, dual_mip_xx, mip_xx, mip_pairs

In [3]:
def correlations(intensities):
    x1h = intensities
    x2h = np.zeros([height, width])
    for i in range(0, height):
        for j in range(0, width-1):
            x2h[i][j] = x1h[i][j+1]
        x2h[i][width-1] = x1h[i][width-1]

    total_N = np.size(x1h)
    mean_x1h = x1h.sum()/total_N
    mean_x2h = x2h.sum()/total_N

    aa1 = (x1h - mean_x1h)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2h - mean_x2h)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1h - mean_x1h)*(x2h - mean_x2h)
    hcov = cc.sum()/total_N

    hcorr = hcov/(std_dev1 * std_dev2)
    print('hcorr', hcorr)

    # Vertical correlation
    x1v = intensities
    x2v = np.zeros([height, width])
    for i in range(0, height-1):
        for j in range(0, width):
            x2v[i][j] = x1v[i+1][j]
            x2v[height-1][j] = x1v[height-1][j]

    total_N = np.size(x1v)
    mean_x1v = x1v.sum()/total_N
    mean_x2v = x2v.sum()/total_N

    aa1 = (x1v - mean_x1v)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2v - mean_x2v)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1v - mean_x1v)*(x2v - mean_x2v)
    vcov = cc.sum()/total_N

    vcorr = vcov/(std_dev1 * std_dev2)
    print('vcorr',vcorr)

    # Diagonal correlation
    x1d = intensities
    x2d = np.zeros([height, width])
    for i in range(0, height-1):
        for j in range(0, width-1):
            x2d[i][j] = x1d[i+1][j+1]
            x2d[height-1][j] = x1d[height-1][j]
        x2d[i][width-1] = x1d[i][width-1]
    x2d[height-1][width-1] = x1d[height-1][width-1]

    total_N = np.size(x1d)
    mean_x1d = x1d.sum()/total_N
    mean_x2d = x2d.sum()/total_N

    aa1 = (x1d - mean_x1d)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2d - mean_x2d)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1d - mean_x1d)*(x2d - mean_x2d)
    dcov = cc.sum()/total_N

    dcorr = dcov/(std_dev1 * std_dev2)
    print('dcorr',dcorr)
    
    return hcorr, vcorr, dcorr

In [4]:
def eve_distortion(eve_intensities): #from the reconstructed image in uint8 type
    feve_intensities = eve_intensities.astype(np.float)
    fintensities = intensities.astype(np.float)
    dif = (fintensities -  feve_intensities)
    eve_dist_mat = dif**2
    occurance_sum = height*width #?!
    eve_dist = eve_dist_mat.sum()/occurance_sum
    return eve_dist

In [5]:
img_file = 'lena_gray_512.tif'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)

all_nq_ac = []
for i in range(0,int(height/8)):
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_ac = np.delete(nq_coef, [0,0])
        all_nq_ac = np.append(all_nq_ac, nq_coef_ac)
        
nq_u, nq_counts = np.unique(all_nq_ac, return_counts=True)

In [7]:
'''this cell takes time to finish'''

#nq category
all_var = {}
all_prbs = {}
all_prb_whole_cat = {}
all_pairs_by_index = {}
all_dct_distortions = {}
all_dct_entropies = {}
for size in range(1,9):

    cells_num = 2**size

    a0 = -2**size + 1
    a1 = -2**(size-1)
    a2 =  2**(size-1)
    a3 =  2**(size)-1

    a_set1 = np.array(range(a0, a1+1))
    a_set2 = np.array(range(a2, a3+1))
    a_set = np.append(a_set1, a_set2)

    nq_counts_cat = np.zeros(cells_num)
    for i in a_set1:
        if i in nq_u:
            nq_counts_cat[i+(cells_num-1)] =  nq_counts[np.argwhere(nq_u == i)]

    for i in a_set2:
        if i in nq_u:
            nq_counts_cat[i] =  nq_counts[np.argwhere(nq_u == i)]

    var = a_set
    sum_counts = sum(nq_counts_cat)
    prbs = nq_counts_cat/sum_counts

    prb_whole_cat = sum_counts/sum(nq_counts)
    
    all_var[size]=var
    all_prbs[size]=prbs
    all_prb_whole_cat[size]=prb_whole_cat
    
    # MIP #######################################################################
    all_yijs = []
    all_dijs = []
    all_prb0s = []
    for i in range(0,cells_num-1):
        for j in range(i+1,cells_num):

            '''Choose the strategy'''
    # # Strategy1: Eve knows the source statistics which is rare in image encrytion
    #         possible_yij = (var[i]*prbs[i]+var[j]*prbs[j])/(prbs[i]+prbs[j])

    # # # Strategy2: Eve knows the reconstruction value of each cell
    # #         possible_yij = (var[i]+var[j])/2
    # #         #possibel_yij = random.choice([var[i],var[j]])



    #         all_yijs = np.append(all_yijs, possible_yij)

    #         possible_dij\
    #         = var[i]**2*prbs[i] + possible_yij**2*prbs[i] - possible_yij*2*var[i]*prbs[i]\
    #         + var[j]**2*prbs[j] + possible_yij**2*prbs[j] - possible_yij*2*var[j]*prbs[j]

    # Strategy3
            yi = var[i]
            yj = var[j]
            possible_dij = 0.5*(
              var[i]**2*prbs[i] + yi**2*prbs[i] - yi*2*var[i]*prbs[i]\
            + var[j]**2*prbs[j] + yi**2*prbs[j] - yi*2*var[j]*prbs[j]\
            + var[i]**2*prbs[i] + yj**2*prbs[i] - yj*2*var[i]*prbs[i]\
            + var[j]**2*prbs[j] + yj**2*prbs[j] - yj*2*var[j]*prbs[j])


            all_dijs = np.append(all_dijs, possible_dij)

            possible_prb0 = max(prbs[j],prbs[i])
            all_prb0s = np.append(all_prb0s, possible_prb0)

    ######### ######### ######### ######### ######### ######### ######### #########
    '''For distortion and entropy, we must implement the integer linear programming.'''
    '''Linear programming does not results in binary values'''

    all_length = int(cells_num * (cells_num-1) / 2)

    A_eq = np.zeros([cells_num, all_length])
    k = 0
    for i in range(0, cells_num-1):
        for j in range(i+1,cells_num):
            A_eq[i,k]=1
            k+=1
    k = 0
    for j in range(1, cells_num):
        for i in range (j,cells_num):
            A_eq[i,k]=1
            k+=1

    b_eq = np.ones([cells_num,])
    
    
    #MIP 1
#     results_all_lambdas = []
#     for Lambda in [0, 1000, 1000000]:
#         results = mip_entropy_constrained(Lambda)

    Lambda = 0
    results = mip_entropy_constrained(Lambda) 
    all_pairs_by_index[size]= results[0]
    all_dct_distortions[size]= results[1]
    all_dct_entropies[size]= results[2]


1_DE(dB)_lam0 3.010299956639812
1_H(S0)_lam0 0.9999742372770573

1_DE(dB)_lam0 10.969100130080562
1_H(S0)_lam0 0.9742285540464419

1_DE(dB)_lam0 17.817553746524688
1_H(S0)_lam0 0.892405376490351

1_DE(dB)_lam0 24.224256763712045
1_H(S0)_lam0 0.8991718579823471

1_DE(dB)_lam0 30.43219724415456
1_H(S0)_lam0 0.9170765127256347

1_DE(dB)_lam0 36.555616720915594
1_H(S0)_lam0 0.9145040666550277

1_DE(dB)_lam0 42.6688555186075
1_H(S0)_lam0 0.8889317402498766

1_DE(dB)_lam0 48.8177528242392
1_H(S0)_lam0 0.6982420008020692


In [13]:
# At Eve #
eve_msb0_intensities = np.zeros([height,width])
eve_msb1_intensities = np.zeros([height,width])
random.seed(0)

for i in range(0,int(height/8)):
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_ac = np.delete(nq_coef, [0,0])
        nq_coef_dc = nq_coef[0,0]
        
        eve_ac_msb0 = nq_coef_ac.copy()
        eve_ac_msb1 = nq_coef_ac.copy()
        
        for m in range(np.size(nq_coef_ac)):
            org_coef = nq_coef_ac[m]
            
            if org_coef != 0 and org_coef < 256 and org_coef > -256:
                coef_size = np.floor(np.log2(abs(org_coef)))+1
                prbs_in_size = all_prbs[coef_size]
                vars_in_size = all_var[coef_size]

                pairs_by_index = all_pairs_by_index[coef_size]
                first_half = pairs_by_index[0]-(2**coef_size-1)
                second_half = pairs_by_index[1]

                if org_coef in first_half:
                    position = np.argwhere(first_half == org_coef)
                    paired_coef = second_half[position]

                if org_coef in second_half:
                    position = np.argwhere(second_half == org_coef)
                    paired_coef = first_half[position]

                org_position_in_vars = np.argwhere(vars_in_size == org_coef)
                #why is int necessary?!
                paired_position_in_vars = np.argwhere(vars_in_size == int(paired_coef))

                org_prb = prbs_in_size[org_position_in_vars]
                paired_prb = prbs_in_size[paired_position_in_vars]
                ''' Choose the way of assigning zero to MSBs'''
                # Considering Entropy
                # eve_msb0
                if org_prb < paired_prb:
                    eve_ac_msb0[m]= paired_coef

                # eve_msb1
                if org_prb > paired_prb:
                    eve_ac_msb1[m]= paired_coef
                    
                    
#                 # Random 0 assignment
#                 paired_coeffs = [org_coef, paired_coef]
#                 random_choice = random.randint(0,1)
#                 eve_ac_msb0[m] = paired_coeffs[random_choice]               
#                 remained_coeff = np.delete(paired_coeffs, random_choice)
#                 eve_ac_msb1[m] = remained_coeff
                
        eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, nq_coef_dc)
        eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])
        
        eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
        eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
        eve_msb0_blk = np.round(eve_msb0_blk2)  
        eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk
        
        
        eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, nq_coef_dc)
        eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])
        
        eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
        eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
        eve_msb1_blk = np.round(eve_msb1_blk2)  
        eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk

In [9]:
eve_msb0_uint8 =eve_msb0_intensities.astype(np.uint8)
cv2.imshow('eve_msb0', eve_msb0_uint8)

''' Choose the way of assigning zero to MSBs'''
# filename = 'eve_msb0_lambda0_EC.png'
filename = 'eve_msb0_lambda0_Random.png'
cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb0_uint8)
# cv2.imwrite(filename, eve_msb0_uint8)
cv2.waitKey(1000)

eve_msb1_uint8 =eve_msb1_intensities.astype(np.uint8)
cv2.imshow('eve_msb1', eve_msb1_uint8)

''' Choose the way of assigning zero to MSBs'''
# filename = 'eve_msb1_lambda0_EC.png'
filename = 'eve_msb1_lambda0_Random.png'
cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb1_uint8)
# cv2.imwrite(filename, eve_msb1_uint8)
cv2.waitKey(1000)

-1

In [10]:
cv2.destroyAllWindows()

In [12]:
# random msb assignment
eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_intensities))
eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_intensities))
print(eve_msb0_dist_db)
print(eve_msb1_dist_db)

27.116803328682245
27.245386711639348


In [14]:
#entropy constrained
eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_intensities))
eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_intensities))
print(eve_msb0_dist_db)
print(eve_msb1_dist_db)

24.171778412301204
28.726028411425453
